In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd drive/My Drive/Colab Notebooks/ジャンクション/顔検出/darknet

/content/drive/My Drive/Colab Notebooks/ジャンクション/顔検出/darknet


In [37]:
ls

wider_face_split/  WIDER_train.zip  WIDER_val/


In [0]:
#widerのデータセットを入れてなかったとき実行
import zipfile
with zipfile.ZipFile('WIDER_train.zip') as existing_zip:
    existing_zip.extractall('WIDER_train')

#Darknetのインストール及び、Darknet　Yolo v3をWIDER FACEデータセットで学習させ、重みファイルを作成する<br>
参考サイト=https://qiita.com/ha9kberry/items/e7c822dd0874e5a24639

#Darknetのインストール

In [0]:
#githubからclone
!git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 8124 (delta 6), reused 5 (delta 1), pack-reused 8102
Receiving objects: 100% (8124/8124), 8.49 MiB | 3.91 MiB/s, done.
Resolving deltas: 100% (5441/5441), done.
Checking out files: 100% (1926/1926), done.


In [0]:
cd darknet

/content/drive/My Drive/Colab Notebooks/ジャンクション/顔検出/darknet/darknet


In [0]:
#MakeFile編集（GPU=1、CUDNN=1に変更）（coraboratoryじゃなくて直接書き換えました）
!vi Makefile

/bin/bash: vi: command not found


In [0]:
#コンパイル
!make

#実行
!./darknet
# 下記が出力されればOK
#usage: ./darknet <function>

#顔検出出来るようDarknetを学習させる前準備

In [0]:
#Darknetで使うImageNetの重みをダウンロード
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2019-01-05 23:42:05--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.3.39
Connecting to pjreddie.com (pjreddie.com)|128.208.3.39|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  42.3MB/s    in 3.7s    

2019-01-05 23:42:09 (41.6 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**学習に使う設定ファイル作成<br>
ファイル名:obj.data<br>
ディレクトリ:cfg/obj.data**<br>
classes：クラス数<br>
train：trainデータ画像のパス一覧ファイル<br>
valid：testデータ画像のパス一覧ファイル<br>
names：クラス名一覧ファイル<br>
backup：学習後のweightファイル保存先<br>



In [0]:
#obj.dataに下記内容をコメントアウトを外して追記
"""
classes = 1
train  = face/train.txt
valid  = face/test.txt
names = cfg/obj.names
backup = backup/
"""


**学習に使う設定ファイル作成<br>
ファイル名:obj.name<br>
ディレクトリ:cfg/obj.name**<br>

In [0]:
#obj.nameに下記内容をコメントアウトを外して追記
"""
face
"""

**学習に使う設定ファイル編集（二箇所）<br>
ファイル名:yolov3.cfg<br>
ディレクトリ:cfg/yolov3.cfg**<br>

In [0]:
#yolov3.cfgの編集（二箇所）
"""
Line 3：コメントアウト
Line 4：コメントアウト
Line 6：batch=24に設定
Line 7：subdivisions=8に設定
"""
"""
[net]
# Testing
# batch=1
# subdivisions=1
# Training
batch=24
subdivisions=8
...
"""

"""
filters=(classes+5)*3
Line 603：filters=18に設定
Line 610：classes=1に設定
Line 689：filters=18に設定
Line 696：classes=1に設定
Line 776：filters=18に設定
Line 783：classes=1に設定
"""
"""
...
[convolutional]
size=1
stride=1
pad=1
filters=18
activation=linear


[yolo]
mask = 0,1,2
anchors = 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326
classes=1
num=9
jitter=.3
ignore_thresh = .7
truth_thresh = 1
random=1
"""

**ディレクトリ構成***
前回作成したデータセット（face）をdarknet直下に格納<br>
darknet/<br>
  ┣ backup/<br>
  ┣ cfg/<br>
  ┃  ┣ yolov3.cfg<br>
  ┃  ┣ obj.data<br>
  ┃  ┣ obj.name<br>
  ┃  ...<br>
  ┣ face/<br>
  ...<br>
  ┣ darknet<br>
  ┣ Makefile<br>
  ┣ darknet53.conv.74<br>
  ...

#顔検出出来るようDarknetを学習させる

**100iterationsごとにbackup直下に重みweightsファイルが作成される**<br>
参考サイトはコンソール上の学習過程を見て、<br>
avg lossに変化が見られなくなったので強制終了した

In [0]:
#学習の実行
!./darknet detector train cfg/obj.data cfg/yolov3.cfg darknet53.conv.74